<a href="https://colab.research.google.com/github/Abhi23run/Uncertainty_Quantification_LLMs/blob/main/Uncertainty_Quantification_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
dir_path = '/content/drive/MyDrive/Uncertainty_Quantification'
os.chdir(dir_path)

In [3]:
%ls

data_normal.csv             Sample_data_creation_stress_index.ipynb
data_with_stress_index.csv  Uncertainty_Quantification_llama.ipynb


In [4]:
%%capture
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1
!pip install torch

In [5]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [6]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token

Enter your Hugging Face token: ··········


In [7]:
# model_id =  "NousResearch/Llama-2-7b-hf"
# model_id = "meta-llama/Llama-2-7b-chat-hf"
model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",token=hf_token)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id,token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
# from transformers import StoppingCriteria
# class EosListStoppingCriteria(StoppingCriteria):
#     def __init__(self, eos_sequence = [319,350,360,315]):
#         self.eos_sequence = eos_sequence

#     def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
#         last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
#         return self.eos_sequence in last_ids

In [12]:
import warnings
warnings.filterwarnings('ignore')

prompt = f"""
###Instruction: Act as a science expert and help in answering the following multiple choice question. The answer should be one of the option choices.
Follow the pattern from the first example and answer the second question accordingly.

###Question:
This is a question from high school biology.A piece of potato is dropped into a beaker of pure water.
Which of the following describes the activity after the potato is immersed into the water?
(A) Water moves from the potato into the surrounding water.
(B) Water moves from the surrounding water into the potato.
(C) Potato cells plasmolyze.
(D) Solutes in the water move into the potato.

###Answer:
B

###Question:
You are the world’s best expert in high school biology. From the solubility rules, which of the following is true?
(A) All hydroxides are soluble
(B) All sulfates are soluble
(C) All chlorides, bromides, and iodides are soluble
(D) All ammonium-containing compounds are soluble

###Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=2,
    )[0],
    skip_special_tokens=False
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
<s> 
###Instruction: Act as a science expert and help in answering the following multiple choice question. The answer should be one of the option choices. 
Follow the pattern from the first example and answer the second question accordingly.

###Question:
This is a question from high school biology.A piece of potato is dropped into a beaker of pure water. 
Which of the following describes the activity after the potato is immersed into the water?
(A) Water moves from the potato into the surrounding water.
(B) Water moves from the surrounding water into the potato.
(C) Potato cells plasmolyze.
(D) Solutes in the water move into the potato.

###Answer:
B

###Question:
You are the world’s best expert in high school biology. From the solubility rules, which of the following is true?
(A) All hydroxides are soluble
(B) All sulfates are soluble
(C) All chlorides, br

In [9]:
prompt = f"""
###Instruction: Act as a science expert and help in answering the following multiple choice question. The answer should be one of the option choices.
Follow the pattern from the first example and answer the second question accordingly.

###Question:
This is a question from high school biology.A piece of potato is dropped into a beaker of pure water.
Which of the following describes the activity after the potato is immersed into the water?
(A) Water moves from the potato into the surrounding water.
(B) Water moves from the surrounding water into the potato.
(C) Potato cells plasmolyze.
(D) Solutes in the water move into the potato.

###Answer:
B

###Question:
You are the world’s best expert in high school biology. From the solubility rules, which of the following is true?
(A) All hydroxides are soluble
(B) All sulfates are soluble
(C) All chlorides, bromides, and iodides are soluble
(D) All ammonium-containing compounds are soluble

###Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
outputs=model(inputs["input_ids"])

next_token_logits = outputs.logits[:, -1]

In [10]:
next_token_logits

tensor([[-10.2891, -10.4609,   5.5469,  ...,  -6.3281,  -6.3789,  -3.5430]],
       grad_fn=<SelectBackward0>)

In [12]:
def softmax(x):
  e_x=np.exp(x-np.max(x))
  return e_x/e_x.sum()


tokens_of_interest=['A','B','C','D']
token_indices = tokenizer.convert_tokens_to_ids(tokens_of_interest)
indices_in_logits = {token: next_token_logits[0,token_idx].item() for token, token_idx in zip(tokens_of_interest, token_indices)}

label_to_softmax_dict=dict(zip(tokens_of_interest,softmax(np.array(list(indices_in_logits.values())))))

print(label_to_softmax_dict)

In [14]:
indices_in_logits

{'A': 21.578125, 'B': 20.78125, 'C': 21.8125, 'D': 21.5}

In [32]:
label_to_softmax_dict

{'A': 0.274747724454225,
 'B': 0.12383850168956001,
 'C': 0.347313666960872,
 'D': 0.25410010689534285}

In [9]:
df_stress_index=pd.read_csv('data_with_stress_index.csv',usecols=lambda col:col not in ['Unnamed: 0'] )

In [10]:
df_stress_index

,news_sentiment,broker_count_imbalance,volume_indicator,benchmark_price_difference,trade_count_imbalance,one_sided_trade_indicator,tranche_size_indicator,stress_index
0,0.393144,0.124362,0.677889,0.497776,-0.720698,-0.680076,0.724869,0.551515
1,0.772496,0.369376,-0.877238,0.139748,0.851864,-0.257519,0.586063,0.623805
2,-0.189353,0.190013,-0.485352,-0.364914,-0.183833,-0.235899,0.356616,0.305677
3,0.219090,-0.577930,0.876170,0.495462,0.182495,0.094340,0.106455,0.599768
4,0.038766,0.474529,0.345526,-0.153370,-0.020692,0.238055,0.432594,0.594587
...,...,...,...,...,...,...,...,...
4995,-0.502263,0.409759,-0.087573,0.323973,-0.960938,0.359897,0.595025,0.439501
4996,-0.444850,-0.969700,0.050369,-0.105244,0.213531,0.319491,0.455737,0.360712
4997,0.520867,0.304688,-0.802427,0.362181,-0.445552,0.071510,0.163793,0.444237
4998,-0.251193,0.288159,-0.617303,0.445843,0.956045,0.024746,0.470942,0.589725


Bucketing the stress index into 10 different segments (option choices)

In [12]:
[str({i:j for i,j in row.to_dict().items() if i not in ['stress_index','stress_index_bucket']}) for _, row in df_stress_index.head(2).iterrows()]

["{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}",
 "{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0.8772379830243947, 'benchmark_price_difference': 0.1397481154484952, 'trade_count_imbalance': 0.8518638934191982, 'one_sided_trade_indicator': -0.2575187071551993, 'tranche_size_indicator': 0.5860627411447076}"]

In [11]:
df_stress_index.loc[:,'stress_index_bucket']=pd.cut(df_stress_index['stress_index'],bins=10,labels=[str(np.round(x,1))+'-'+str(np.round(x+0.1,1)) for x in np.arange(0,1,0.1)])

In [12]:
df_stress_index

,news_sentiment,broker_count_imbalance,volume_indicator,benchmark_price_difference,trade_count_imbalance,one_sided_trade_indicator,tranche_size_indicator,stress_index,stress_index_bucket
0,0.393144,0.124362,0.677889,0.497776,-0.720698,-0.680076,0.724869,0.551515,0.5-0.6
1,0.772496,0.369376,-0.877238,0.139748,0.851864,-0.257519,0.586063,0.623805,0.6-0.7
2,-0.189353,0.190013,-0.485352,-0.364914,-0.183833,-0.235899,0.356616,0.305677,0.3-0.4
3,0.219090,-0.577930,0.876170,0.495462,0.182495,0.094340,0.106455,0.599768,0.5-0.6
4,0.038766,0.474529,0.345526,-0.153370,-0.020692,0.238055,0.432594,0.594587,0.5-0.6
...,...,...,...,...,...,...,...,...,...
4995,-0.502263,0.409759,-0.087573,0.323973,-0.960938,0.359897,0.595025,0.439501,0.4-0.5
4996,-0.444850,-0.969700,0.050369,-0.105244,0.213531,0.319491,0.455737,0.360712,0.3-0.4
4997,0.520867,0.304688,-0.802427,0.362181,-0.445552,0.071510,0.163793,0.444237,0.4-0.5
4998,-0.251193,0.288159,-0.617303,0.445843,0.956045,0.024746,0.470942,0.589725,0.5-0.6


**Trading Bot Prompt**

In [13]:
import random
random.seed(42)

In [14]:
def shuffle_list(original_list):
    # Create a copy of the list
    shuffled_list = original_list.copy()

    # Shuffle the copy
    random.shuffle(shuffled_list)

    return shuffled_list

In [15]:
list_stress_index_bucket=[str(np.round(x,1))+'-'+str(np.round(x+0.1,1)) for x in np.arange(0,1,0.1)]
list_option_choices=[chr(x) for x in range(65,65+len(list_stress_index_bucket))]

In [102]:
# Note : Act as a finance expert and help in answering the following multiple choice question. The answer should be one of the option choices.
# Follow the pattern from the first example and answer the second question accordingly.

In [16]:
instruction='''
###Instruction:
You are fixed income corporate bond trader at BlackRock. Your role is to analyse the data that I will provide to you and by analysing the data you need to select the correct option for classifying the stress index (has a range of 0 to 1) into one of the defined buckets.
The stress index is calculated using the features as defined below :

- news_sentiment: that means sentiment of the news published for the bond issuer in last 7 days in a scale of -1 to +1, -1 means negative sentiment about the security and +1 means positive news about the security.
- broker_count_imbalance: that means count difference of brokers trying to buy or sell a security in the same side in the last 2 days. -1 means too many brokers trying to sell the security and +1 means too many brokers trying to buy the security.
- volume_indicator: that means a bond being heavily bought and sold in last 2 days, -1 means a bond being suddenly sold at a very high volume and +1 means a bond being suddenly bought at a very high level.
- benchmark_price_difference: that means how far is the price of a quote for a bond quoted by a broker compared to benchmark price. If a bond is aggressively quoted compared to the benchmark, then it’s a sign of stress for that bond. It is in the range of -1 to +1, -1 means quoted price below the benchmark price and +1 means quoted price is above the benchmark price.
- trade_count_imbalance: that means count difference of actual buy and sell trades of a security in the last 2 days. It is in the range of -1 to +1, -1 means number of sell trades were higher compared to buy trades and +1 means number of buy trades were higher compared to sell trades
- one_sided_trade_indicator: that means count difference of actual buy and sell trades of a security in the same side in the last 7 days. It is in the range of -1 to +1, -1 means too many trades happened at sell side and +1 means too many trades happened in the buy side in the last 7 days
- tranche_size_indicator: If all the trades are in one direction and the amount outstanding of the bond is very high, e.g. $5 billion tranche, then the bond is not stressed, but if the If all the trades are in one direction and the amount outstanding of the bond is very low, e.g. $750 million tranche, then the bond is stressed. It is in the range of 0 to +1, 0 means tranche size is high whereas 1 means tranche size is very low

###Question:
Given the following feature value dict :
{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}

Return the correct option for classifying the stress index from the following options based on the features given above. Do not generate anything else apart from the option.
'''

In [17]:
choice_prompt=''''''
for i,j in zip(list_option_choices,shuffle_list(list_stress_index_bucket)):
  choice_prompt+=f'({i}) {j}\n'

In [18]:
example_prompt=''''''
example_prompt+=instruction
example_prompt+=choice_prompt
example_prompt+='''
###Answer:
E
'''

In [19]:
print(example_prompt)


###Instruction: 
You are fixed income corporate bond trader at BlackRock. Your role is to analyse the data that I will provide to you and by analysing the data you need to select the correct option for classifying the stress index (has a range of 0 to 1) into one of the defined buckets.
The stress index is calculated using the features as defined below : 
 
- news_sentiment: that means sentiment of the news published for the bond issuer in last 7 days in a scale of -1 to +1, -1 means negative sentiment about the security and +1 means positive news about the security.
- broker_count_imbalance: that means count difference of brokers trying to buy or sell a security in the same side in the last 2 days. -1 means too many brokers trying to sell the security and +1 means too many brokers trying to buy the security.
- volume_indicator: that means a bond being heavily bought and sold in last 2 days, -1 means a bond being suddenly sold at a very high volume and +1 means a bond being suddenly b

In [20]:
final_prompt=''''''
final_prompt+=example_prompt
final_prompt+='''
###Question:
Given the following feature value dict :
{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0.8772379830243947, 'benchmark_price_difference': 0.1397481154484952, 'trade_count_imbalance': 0.8518638934191982, 'one_sided_trade_indicator': -0.2575187071551993, 'tranche_size_indicator': 0.5860627411447076}

Return the correct option for classifying the stress index from the following options based on the features given above. Do not generate anything else apart from the option.
'''
choice_prompt=''''''
for i,j in zip(list_option_choices,shuffle_list(list_stress_index_bucket)):
  choice_prompt+=f'({i}) {j}\n'

final_prompt+=choice_prompt
final_prompt+='''
###Answer:
'''

In [21]:
print(final_prompt)


###Instruction: 
You are fixed income corporate bond trader at BlackRock. Your role is to analyse the data that I will provide to you and by analysing the data you need to select the correct option for classifying the stress index (has a range of 0 to 1) into one of the defined buckets.
The stress index is calculated using the features as defined below : 
 
- news_sentiment: that means sentiment of the news published for the bond issuer in last 7 days in a scale of -1 to +1, -1 means negative sentiment about the security and +1 means positive news about the security.
- broker_count_imbalance: that means count difference of brokers trying to buy or sell a security in the same side in the last 2 days. -1 means too many brokers trying to sell the security and +1 means too many brokers trying to buy the security.
- volume_indicator: that means a bond being heavily bought and sold in last 2 days, -1 means a bond being suddenly sold at a very high volume and +1 means a bond being suddenly b

In [24]:
inputs = tokenizer(final_prompt, return_tensors='pt')
generated_ids=model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=2)
output = tokenizer.decode(
    generated_ids[0],
        # stopping_criteria = [EosListStoppingCriteria()] ,
    skip_special_tokens=False
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1482: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
<s> 
###Instruction: 
You are fixed income corporate bond trader at BlackRock. Your role is to analyse the data that I will provide to you and by analysing the data you need to select the correct option for classifying the stress index (has a range of 0 to 1) into one of the defined buckets.
The stress index is calculated using the features as defined below : 
 
- news_sentiment: that means sentiment of the news published for the bond issuer in last 7 days in a scale of -1 to +1, -1 means negative sentiment about the security and +1 means positive news about the security.
- broker_count_imbalance: that means count difference of brokers trying to buy or sell a security in the same side in the last 2 days. -1 means too many brokers trying to sell the security and +1 means too many brokers trying to buy the security.
- volume_indicator: that means a bond being h

In [40]:
print(final_prompt)


###Instruction: 
You are fixed income corporate bond trader at BlackRock. Your role is to analyse the data that I will provide to you and by analysing the data you need to select the correct option for classifying the stress index (has a range of 0 to 1) into one of the defined buckets.
The stress index is calculated using the features as defined below : 
 
- news_sentiment: that means sentiment of the news published for the bond issuer in last 7 days in a scale of -1 to +1, -1 means negative sentiment about the security and +1 means positive news about the security.
- broker_count_imbalance: that means count difference of brokers trying to buy or sell a security in the same side in the last 2 days. -1 means too many brokers trying to sell the security and +1 means too many brokers trying to buy the security.
- volume_indicator: that means a bond being heavily bought and sold in last 2 days, -1 means a bond being suddenly sold at a very high volume and +1 means a bond being suddenly b

**Next Token Logits**

In [22]:
inputs = tokenizer(final_prompt, return_tensors='pt')
# generated_ids=model.generate(
#         inputs["input_ids"].to('cuda'),
#         temperature=0.2,
#         do_sample=True,
#         max_new_tokens=1)

In [24]:
generated_ids.shape

torch.Size([1, 1389])

In [29]:
tokenizer.decode(generated_ids[0,-2])

'D'

In [25]:
torch.cuda.empty_cache()

In [23]:
outputs=model(inputs["input_ids"])
# next_token_logits = outputs.logits[:, -1]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 51.06 MiB is free. Process 31584 has 14.70 GiB memory in use. Of the allocated memory 14.06 GiB is allocated by PyTorch, and 520.85 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def softmax(x):
  e_x=np.exp(x-np.max(x))
  return e_x/e_x.sum()


token_indices = tokenizer.convert_tokens_to_ids(list_option_choices)
indices_in_logits = {token: next_token_logits[0,token_idx].item() for token, token_idx in zip(list_option_choices, token_indices)}

label_to_softmax_dict=dict(zip(tokens_of_interest,softmax(np.array(list(indices_in_logits.values())))))

print(label_to_softmax_dict)

New Prompt

In [ ]:
instruction="""
###Instruction:
You are fixed income corporate bond trader. Use the feature values that I will provide and select the correct option for classifying the stress index (range of 0 to 1) into one of the defined buckets.
Feature descriptions:
- news_sentiment: Scale from -1 (negative) to +1 (positive) reflecting the sentiment in news about the bond issuer over the past 7 days.
- broker_count_imbalance: Difference in broker count buying or selling the same security in the last 2 days, ranging from -1 (more selling) to +1 (more buying).
- volume_indicator: Indicates if a bond is heavily traded in the last 2 days, with -1 for high selling volume and +1 for high buying volume.
- benchmark_price_difference: Compares a bond's quoted price to the benchmark, ranging from -1 (below benchmark) to +1 (above benchmark), indicating price stress.
- trade_count_imbalance: Difference in buy and sell trades of a security over 2 days, from -1 (more sells) to +1 (more buys).
- one_sided_trade_indicator: Imbalance in buy or sell trades over 7 days, with -1 for predominantly sell trades and +1 for buy trades.
- tranche_size_indicator: Assesses bond stress by the direction of trades and tranche size, ranging from 0 (large tranche) to 1 (small tranche).
###Question:
Given the feature value dict as:
{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}
Return the correct option for classifying stress index from the following options based on above features. Do not generate anything else apart from the option.
"""

In [1]:
def lac(true_label, label_softmax_dict):
    lac_score = 1.0 - label_softmax_dict[true_label]
    return lac_score

In [ ]:
def get_values_till_key(sorted_dict, key):
    values_before_key = []
    for k, v in sorted_dict.items():
        if k == key:
            values_before_key.append(v)
            break
        values_before_key.append(v)
    return values_before_key

In [ ]:
def aps(true_label, label_softmax_dict):
    sorted_softmax_dict = dict(sorted(label_to_softmax_dict.items(), key=lambda item: item[1], reverse=True))
    high_labels = get_values_till_key(sorted_softmax_dict, true_label)
    aps_score = sum(high_labels)
    return aps_score